# Meta Classifier
Detect images generated using Meta AI. 

## Setup 

In [7]:
import torch
import os
import sys
import tqdm
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler


import utils_img
import utils

sys.path.append('src')
from loss.loss_provider import LossProvider

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"


In [2]:
import gc 

def clear_memory():
    '''
    Delete unused tensors on GPU to reduce OOM issues. 
    '''
    try:
        del msg_extractor
    except: 
        pass
    try: 
        del img
    except:
        pass
    try: 
        del imgs
    except:
        pass
    try: 
        del targets
    except:
        pass
    gc.collect()
    torch.cuda.empty_cache()

clear_memory()

## Load Data

In [3]:
img_size = 256

# augment training set with some transformations 
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(img_size),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=180),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    ])


In [5]:
from torch.utils.data import DataLoader, Subset
from torchvision.datasets.folder import default_loader
from torch.nn.utils.rnn import pad_sequence
import numpy as np 

batch_size = 4
train_dir = "data/meta/train"
train_size = 4770
val_dir = "data/meta/val"
val_size = 596

class ImageFolder:
    """An image folder dataset intended for supervised learning."""

    def __init__(self, path, transform=None, loader=default_loader):
        self.samples = [x for x in utils.get_image_paths(path) if not "aug" in x]
        self.loader = loader
        self.transform = transform

    def __getitem__(self, idx: int):
        """
        Returns the image with its corresponding label. Images are 
        labeled 0 if the image is not watermarked, else 1. 
        """
        assert 0 <= idx < len(self)
        path = self.samples[idx]
        img = self.loader(path)
        label = 0 if "orig" in path else 1
        if self.transform:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.samples)


def get_dataloader(data_dir, transform, batch_size=128, num_imgs=None, shuffle=False, num_workers=4):
    """ Get dataloader for the images in the data_dir. """
    dataset = ImageFolder(data_dir, transform=transform)
    if num_imgs is not None:
        dataset = Subset(dataset, np.random.choice(len(dataset), num_imgs, replace=False))
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True, drop_last=False, collate_fn=None)


train_loader = get_dataloader(train_dir, train_transform, batch_size, num_imgs=train_size, shuffle=True)

# can try validate with image augmentations (train_transform) or without (val_transform)
val_loader = get_dataloader(val_dir, train_transform, 2, num_imgs=val_size, shuffle=True)


## Create Model 
Create a model by finetuning a 64-bit stable signature watermark extractor. Watermark extractor was trained for 300 epochs with the following command:

`torchrun --nproc_per_node=8 main.py --local_rank 0 --val_dir /home/test2017/ --train_dir /home/train2017/ --output_dir output64 --eval_freq 5   --img_size 256 --num_bits 64  --batch_size 16 --epochs 300   --scheduler CosineLRScheduler,lr_min=1e-6,t_initial=300,warmup_lr_init=1e-6,warmup_t=5  --optimizer Lamb,lr=2e-2   --p_color_jitter 0.0 --p_blur 0.0 --p_rot 0.0 --p_crop 1.0 --p_res 1.0 --p_jpeg 1.0   --scaling_w 0.3 --scale_channels False --attenuation none   --loss_w_type bce --loss_margin 1 
`

Watermark extractor was then whitened to ensure that the output bits more independent and well distributed (refer to stable signature paper). A whitened extractor was generated by running the finetune_ldm_decoder.py code once.

`python finetune_ldm_decoder.py --train_dir data/train --val_dir data/val --batch_size 1 --output_dir key0 --seed 0`


In [28]:
msg_extractor = torch.jit.load("hidden/runpod/checkpoint299_whit.pth").to("cpu")

model = nn.Sequential(
    nn.Sequential(*(list(msg_extractor.children())[:-1])),
    nn.Dropout(p=0.1),
    nn.Linear(in_features=64, out_features=1)
)

for param in model.parameters():
    param.requires_grad = True

optimizer = optim.Adam(model.parameters(), lr=0.001)

model

Sequential(
  (0): Sequential(
    (0): RecursiveScriptModule(
      original_name=HiddenDecoder
      (layers): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(
          original_name=ConvBNRelu
          (layers): RecursiveScriptModule(
            original_name=Sequential
            (0): RecursiveScriptModule(original_name=Conv2d)
            (1): RecursiveScriptModule(original_name=BatchNorm2d)
            (2): RecursiveScriptModule(original_name=GELU)
          )
        )
        (1): RecursiveScriptModule(
          original_name=ConvBNRelu
          (layers): RecursiveScriptModule(
            original_name=Sequential
            (0): RecursiveScriptModule(original_name=Conv2d)
            (1): RecursiveScriptModule(original_name=BatchNorm2d)
            (2): RecursiveScriptModule(original_name=GELU)
          )
        )
        (2): RecursiveScriptModule(
          original_name=ConvBNRelu
          (layers): RecursiveScriptModule(

## Train Model 

In [53]:
def load_ckp(checkpoint_fpath, model, optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer, checkpoint['epoch']

def save_ckp(state, checkpoint_name):
    torch.save(state, checkpoint_name)


model, optimizer, start_epoch = load_ckp("models/meta/checkpoint010.pth", model, optimizer)

In [54]:
start_epoch

10

In [46]:
from sklearn.metrics import confusion_matrix

def train(model, data_loader, dataset_size, criterion, optimizer, scheduler, epoch):
    """
    Train model using the data_loader for 1 epoch. Prints out a confusion matrix,
    loss, and accuracy every 100 steps and at the end of the epoch.
    """
    model.train()
    pred_len = 0
    y_pred = []
    y_true = []
    test_correct = 0

    running_loss = 0.0
    # Iterate over data.
    for step, data in enumerate(tqdm.tqdm(data_loader)):
        clear_memory()
        inputs, labels = data
        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float)
        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            targets = labels.unsqueeze(1)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            preds = torch.sigmoid(outputs).round().cpu().detach().numpy().squeeze()
            y_pred.extend(preds)
            labels = labels.cpu().numpy()
            y_true.extend(labels)
            test_correct += int((preds == labels).sum())
            pred_len += preds.size
            if step%100 == 0:
                print(f'Step: {step}, Loss:  {loss.item():.4f}, Acc: {test_correct/pred_len}')
                cf_matrix = confusion_matrix(y_true, y_pred)
                print(cf_matrix)

        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / dataset_size
    print(f'Epoch: {epoch}, Loss: {epoch_loss:.4f}, Acc: {test_correct/pred_len}')
    cf_matrix = confusion_matrix(y_true, y_pred)
    print(cf_matrix)
    scheduler.step()
    return model 
    
def validate(model, data_loader, dataset_size, criterion):
    """
    Validates model using the data_loader. Prints out a confusion matrix,
    loss, and accuracy every 100 steps and at the end.
    """
    model.eval()
    pred_len = 0
    y_pred = []
    y_true = []
    test_correct = 0

    running_loss = 0.0
    # Iterate over data.
    for step, d in enumerate(tqdm.tqdm(data_loader)):
        clear_memory()
        inputs, labels = d
        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float)
        with torch.no_grad():
            outputs = model(inputs)
            targets = labels.unsqueeze(1)
            loss = criterion(outputs, targets)
            preds = torch.sigmoid(outputs).round().cpu().detach().numpy().squeeze()
            y_pred.extend(preds)
            labels = labels.cpu().numpy()
            y_true.extend(labels)
            test_correct += int((preds == labels).sum())
            pred_len += preds.size
            if step%100 == 0:
                print(f'Step: {step}, Loss:  {loss.item():.4f}, Acc: {test_correct/pred_len}')
                cf_matrix = confusion_matrix(y_true, y_pred)
                print(cf_matrix)

        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / dataset_size
    print(f'Val Loss: {epoch_loss:.4f}, Val Acc: {test_correct/pred_len}')
    cf_matrix = confusion_matrix(y_true, y_pred)
    print(cf_matrix)


def train_model(model, train_loader, train_size, val_loader, val_size, criterion, optimizer, scheduler, num_epochs):
    """
    Runs train and validate for num_epochs. 
    """
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        model = train(model, train_loader, train_size, criterion, optimizer, scheduler, epoch)
        clear_memory()
        validate(model, val_loader, val_size, criterion)
        if epoch%2 == 0:
            print(f"saving checkpoint...")
            checkpoint = {
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict()
            }
            save_ckp(checkpoint, f"models/meta/checkpoint{epoch:03}.pth")

    return model  


In [47]:
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)
lr_sch = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

model = train_model(model, train_loader, train_size, val_loader, val_size, criterion, optimizer, lr_sch, 10)


Epoch 0/9
----------


  0%|▏                                                                                                                                                                     | 1/1193 [00:00<09:27,  2.10it/s]

Step: 0, Loss:  0.1607, Acc: 1.0
[[1 0]
 [0 3]]


  8%|█████████████▉                                                                                                                                                      | 101/1193 [00:26<04:51,  3.75it/s]

Step: 100, Loss:  0.0913, Acc: 0.8762376237623762
[[177  23]
 [ 27 177]]


...

Step: 100, Loss:  0.0003, Acc: 0.9603960396039604
[[ 92   5]
 [  3 102]]


 68%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                     | 202/298 [00:19<00:09, 10.56it/s]

Step: 200, Loss:  0.0016, Acc: 0.9477611940298507
[[180  17]
 [  4 201]]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 298/298 [00:28<00:00, 10.50it/s]

Val Loss: 0.0887, Val Acc: 0.9614093959731543
[[281  19]
 [  4 292]]


### Save model

In [52]:
print(f"saving checkpoint...")
checkpoint = {
    'epoch': 10,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict()
}
save_ckp(checkpoint, f"models/meta/checkpoint010.pth")


saving checkpoint...


In [63]:
#jit_model = torch.jit.script(model)
#torch.jit.save(jit_model, "models/meta_classifier.pt")

In [55]:
validate(model, val_loader, val_size, criterion)

  1%|█▋                                                                                                                                                                     | 3/298 [00:00<00:39,  7.49it/s]

Step: 0, Loss:  0.0188, Acc: 1.0
[[1 0]
 [0 1]]


 35%|█████████████████████████████████████████████████████████                                                                                                            | 103/298 [00:09<00:18, 10.70it/s]

Step: 100, Loss:  0.0388, Acc: 0.9801980198019802
[[99  2]
 [ 2 99]]


 68%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                    | 203/298 [00:19<00:08, 10.78it/s]

Step: 200, Loss:  0.0038, Acc: 0.972636815920398
[[191   8]
 [  3 200]]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 298/298 [00:27<00:00, 10.69it/s]

Val Loss: 0.0940, Val Acc: 0.9664429530201343
[[284  16]
 [  4 292]]


## Validation

Code to test on a single image or a test directory. The output 1 indicates that the image is generated by Meta AI, 0 otherwise. 

In [56]:
model, optimizer, start_epoch = load_ckp("models/meta/checkpoint010.pth", model, optimizer)

In [58]:
from PIL import Image

with torch.no_grad():
    #img_path = "/ssd/watermarks/stable_signature/data/meta-aug/331_jpeg_80_w.png"
    img_path = "/ssd/watermarks/stable_signature/meta.jpeg"
    img = Image.open(img_path)
    img = val_transform(img).unsqueeze(0).to(device)
    model.eval()
    output = model(img)
pred = torch.sigmoid(output).round().cpu().detach().numpy().squeeze()
pred

array(1., dtype=float32)

In [59]:
test_dir = "test"
# test with no image augmentations
test_loader = get_dataloader(test_dir, val_transform, 2, num_imgs=500, shuffle=True)
validate(model, test_loader, val_size, criterion)

  1%|██                                                                                                                                                                     | 3/250 [00:00<00:34,  7.11it/s]

Step: 0, Loss:  0.0014, Acc: 1.0
[[2]]


 41%|███████████████████████████████████████████████████████████████████▉                                                                                                 | 103/250 [00:09<00:13, 10.81it/s]

Step: 100, Loss:  0.0131, Acc: 0.9900990099009901
[[ 97   0]
 [  2 103]]


 81%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                               | 203/250 [00:19<00:04, 10.75it/s]

Step: 200, Loss:  0.0265, Acc: 0.9950248756218906
[[200   0]
 [  2 200]]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:23<00:00, 10.67it/s]

Val Loss: 0.0410, Val Acc: 0.99
[[250   2]
 [  3 245]]
